In [2]:
#!/usr/bin/env python

import carla
import random
import time
import numpy as np

class SelfDrivingAgent:
    def __init__(self):
        # Connect to the client
        self.client = carla.Client('localhost', 2000)
        self.client.set_timeout(10.0)

        # Get the world
        self.world = self.client.get_world()
        self.blueprint_library = self.world.get_blueprint_library()
        self.map = self.world.get_map()

        # Set up vehicle
        self.vehicle = None
        self.sensors = {}

    def setup_vehicle(self):
        # Get a random vehicle blueprint
        vehicle_bp = random.choice(self.blueprint_library.filter('vehicle.*'))

        # Get a spawn point
        spawn_points = self.map.get_spawn_points()
        spawn_point = random.choice(spawn_points)

        # Spawn the vehicle
        self.vehicle = self.world.spawn_actor(vehicle_bp, spawn_point)
        print(f"Vehicle spawned: {self.vehicle.type_id}")

    def setup_sensors(self):
        # Set up RGB camera
        camera_bp = self.blueprint_library.find('sensor.camera.rgb')
        camera_bp.set_attribute('image_size_x', '800')
        camera_bp.set_attribute('image_size_y', '600')
        camera_transform = carla.Transform(carla.Location(x=1.5, z=2.4))
        self.sensors['camera'] = self.world.spawn_actor(
            camera_bp,
            camera_transform,
            attach_to=self.vehicle
        )
        self.sensors['camera'].listen(lambda image: self.process_camera_data(image))

        # Set up LiDAR
        lidar_bp = self.blueprint_library.find('sensor.lidar.ray_cast')
        lidar_bp.set_attribute('range', '50')
        lidar_transform = carla.Transform(carla.Location(z=2.0))
        self.sensors['lidar'] = self.world.spawn_actor(
            lidar_bp,
            lidar_transform,
            attach_to=self.vehicle
        )
        self.sensors['lidar'].listen(lambda point_cloud: self.process_lidar_data(point_cloud))

    def process_camera_data(self, image):
        # Convert to numpy array
        array = np.frombuffer(image.raw_data, dtype=np.dtype("uint8"))
        array = np.reshape(array, (image.height, image.width, 4))
        array = array[:, :, :3]  # RGBA to RGB

        # Process image for perception
        # TODO: Implement computer vision algorithms here

    def process_lidar_data(self, point_cloud):
        # Convert to numpy array
        data = np.copy(np.frombuffer(point_cloud.raw_data, dtype=np.dtype('f4')))
        data = np.reshape(data, (int(data.shape[0] / 4), 4))

        # Process point cloud for object detection and mapping
        # TODO: Implement point cloud processing here

    def plan_path(self):
        # Path planning logic
        # TODO: Implement path planning algorithms
        pass

    def control_vehicle(self):
        # Implement vehicle control logic
        control = carla.VehicleControl()
        control.throttle = 0.5  # Example values
        control.steer = 0.0
        control.brake = 0.0
        self.vehicle.apply_control(control)

    def run(self):
        try:
            self.setup_vehicle()
            self.setup_sensors()

            # Control loop
            while True:
                # Perception
                # Planning
                self.plan_path()
                # Control
                self.control_vehicle()

                time.sleep(0.1)  # Control frequency

        finally:
            # Clean up
            for sensor in self.sensors.values():
                sensor.destroy()
            if self.vehicle:
                self.vehicle.destroy()
            print("All actors destroyed.")

if __name__ == "__main__":
    try:
        agent = SelfDrivingAgent()
        agent.run()
    except KeyboardInterrupt:
        print("Simulation stopped by user")

Vehicle spawned: vehicle.citroen.c3
All actors destroyed.
Simulation stopped by user


In [5]:
import carla
client = carla.Client('localhost', 2000)
world = client.get_world()
client.load_world('Town05')

In [7]:
spectator = world.get_spectator()

transform = spectator.get_transform()

location = transform.location
rotation = transform.rotation

spectator.set_transform(carla.Transform())

In [10]:
import random
# Get the blueprint library and filter for the vehicle blueprints
vehicle_blueprints = world.get_blueprint_library().filter('*vehicle*')

# Get the map's spawn points
spawn_points = world.get_map().get_spawn_points()

# Spawn 50 vehicles randomly distributed throughout the map
# for each spawn point, we choose a random vehicle from the blueprint library
for i in range(0,50):
    world.try_spawn_actor(random.choice(vehicle_blueprints), random.choice(spawn_points))

In [11]:

# To train an autonomous agent we need to simulate a the vehicle that it the autonomous agent will control
ego_vehicle = world.spawn_actor(random.choice(vehicle_blueprints), random.choice(spawn_points))

In [12]:
# Create a transform to place the camera on top of the vehicle
camera_init_trans = carla.Transform(carla.Location(z=1.5))

# We create the camera through a blueprint that defines its properties
camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')

# We spawn the camera and attach it to our ego vehicle
camera = world.spawn_actor(camera_bp, camera_init_trans, attach_to=ego_vehicle)

In [13]:
# Start camera with PyGame callback
camera.listen(lambda image: image.save_to_disk('out/%06d.png' % image.frame))

In [14]:
for vehicle in world.get_actors().filter('*vehicle*'):
    vehicle.set_autopilot(True)

In [15]:

ego_bp = world.get_blueprint_library().find('vehicle.lincoln.mkz_2020')

ego_bp.set_attribute('role_name', 'hero')

ego_vehicle = world.spawn_actor(ego_bp, random.choice(spawn_points))


In [16]:
camera.is_listening = False

In [17]:
camera